In [1]:
import requests
from pprint import pprint
import pandas as pd
import zipfile
import io
import json
import os
os.makedirs("results/inputs_extracted_xml", exist_ok=True)
year = "2024"

# Source = "https://www.prix-carburants.gouv.fr/rubrique/opendata/" (site gouvernemental)
url = f"https://donnees.roulez-eco.fr/opendata/annee/{year}"

response = requests.get(url)
print(response)

zip_file = zipfile.ZipFile(io.BytesIO(response.content))
file_name = zip_file.namelist()[0]
print(file_name)

with zipfile.ZipFile(io.BytesIO(response.content), 'r') as zip_ref:
    zip_ref.extractall("results/inputs_extracted_xml")
print(file_name, "extracted")

<Response [200]>
PrixCarburants_annuel_2024.xml
PrixCarburants_annuel_2024.xml extracted


In [2]:
import xml.etree.ElementTree as ET

tree = ET.parse(f"results/inputs_extracted_xml/PrixCarburants_annuel_{year}.xml")

# Obtenir la racine du fichier XML
root = tree.getroot()

data = []
for pdv in root.findall("pdv"):
    station = {
        "id": pdv.get("id"),
        "latitude": pdv.get("latitude"),
        "longitude": pdv.get("longitude"),
        "cp": pdv.get("cp"),
        "ville": pdv.find("ville").text,
        "adresse": pdv.find("adresse").text,
        # "services": [s.text for s in pdv.find("services")],
        "prix": [
            {
                "nom": p.get("nom"),
                "maj": p.get("maj"),
                "valeur": float(p.get("valeur")) if p.get("valeur") is not None else None
            }
            for p in pdv.findall("prix")
        ],
    }
    data.append(station)

# Convertir en JSON
json_data = json.dumps(data, indent=4, ensure_ascii=False)

# Sauvegarder si besoin
os.makedirs("results/inputs_json", exist_ok=True)
with open(f"results/inputs_json/PrixCarburants_annuel_{year}.json", "w", encoding="utf-8") as f:
    f.write(json_data)

print(json_data[:50])  # Afficher un aperçu

[
    {
        "id": "1000001",
        "latitude
